# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 200  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                144       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 2)                 34 

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/200
3122/3122 - 11s - loss: 16038.1045 - val_loss: 1863.9419 - 11s/epoch - 3ms/step
Epoch 2/200
3122/3122 - 2s - loss: 772.9982 - val_loss: 289.7737 - 2s/epoch - 786us/step
Epoch 3/200
3122/3122 - 2s - loss: 148.1849 - val_loss: 64.4407 - 2s/epoch - 784us/step
Epoch 4/200
3122/3122 - 2s - loss: 38.7038 - val_loss: 25.6113 - 2s/epoch - 778us/step
Epoch 5/200
3122/3122 - 3s - loss: 21.9431 - val_loss: 20.5511 - 3s/epoch - 801us/step
Epoch 6/200
3122/3122 - 3s - loss: 19.2808 - val_loss: 18.9584 - 3s/epoch - 845us/step
Epoch 7/200
3122/3122 - 3s - loss: 18.0164 - val_loss: 17.7707 - 3s/epoch - 981us/step
Epoch 8/200
3122/3122 - 2s - loss: 17.0390 - val_loss: 17.1270 - 2s/epoch - 774us/step
Epoch 9/200
3122/3122 - 3s - loss: 16.3076 - val_loss: 16.2151 - 3s/epoch - 818us/step
Epoch 10/200
3122/3122 - 3s - loss: 15.7561 - val_loss: 15.7561 - 3s/epoch - 801us/step
Epoch 11/200
3122/3122 - 3s - loss: 15.3761 - val_loss: 15.4742 - 3s/epoch - 918us/step
Epoch 12/200
3122/3122 - 3s - los

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [6]:
fit = np.load("data/fit.npz")

In [7]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)